<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado desde 2017-2 al 2025-2 por Equipo Docente IIC2233</font>
</p>

# Tabla de contenidos
1. [Multiherencia en Python](#multiherencia-en-python)
    1. [Solucionando el problema del diamante](#solucionando-el-problema-del-diamante)
        1. [Solución](#solución)
        2. [Obteniendo el orden de herencia: el método `__mro__` ](#obteniendo-el-orden-de-herencia-el-método-__mro__)
    2. [Ejemplo multiherencia: clase `Academico`](#ejemplo-multiherencia-clase-academico)
        1. [Solución: uso de `*args` y `**kwargs`](#solución-uso-de-args-y-kwargs)
        2. [Ejemplo: solución para clase `Académico`](#ejemplo-solución-para-clase-académico)
    3. [Ejemplo multiherencia: clase `Avatar`](#ejemplo-multiherencia-clase-avatar)

# Multiherencia en Python

Como vimos en el Notebook "3-multiherencia.ipynb" , tal como es posible que una subclase herede datos y comportamiento de una superclase, también es posible heredar de más de una clase a la vez. Esto se conoce en OOP como **multiherencia**.

Volvamos a ver la clase `Academico`. Un académico posee dos roles: investigador y docente, por lo tanto en nuestro modelo reflejaremos este hecho haciendo que `Academico` herede de las clases ya existentes `Investigador` y `Docente`. Una manera **correcta** de implementar esto en Python sería:

In [1]:
class Investigador:

    def __init__(self, area='', **kwargs):
        # Utilizamos super() para heredar correctamente
        super().__init__(**kwargs)
        self.area = area
        self.num_publicaciones = 0


class Docente:

    def __init__(self, departamento='', **kwargs):
        # Utilizamos super() para heredar correctamente
        super().__init__(**kwargs)
        self.departamento = departamento
        self.num_cursos = 3

# Aquí decimos que Academico hereda tanto de Docente como de Investigador
class Academico(Docente, Investigador):
    
    def __init__(self, nombre, oficina, **kwargs):
        # Utilizamos super() para heredar correctamente
        super().__init__(**kwargs)
        self.nombre = nombre
        self.oficina = oficina


p1 = Academico(
    "Emilia Donoso",
    oficina="O5",
    area="Inteligencia de Máquina",
    departamento="Ciencia De La Computación"
)
print(p1.nombre)
print(p1.area)
print(p1.departamento)

Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación


Podemos ver que para esta la implementación estamos utilizando tanto `super()` como `**kwargs` de maneras que no hemos visto antes y que serán explicadas a lo largo de este notebook, pero para llegar a esto, comenzaremos con una **implementación incorrecta**, pero que usa solamente los contenidos vistos hasta el momento, y la repararemos paso a paso. Esta implementación es:

In [2]:
class Investigador:

    def __init__(self, area):
        self.area = area
        self.num_publicaciones = 0


class Docente:

    def __init__(self, departamento):
        self.departamento = departamento
        self.num_cursos = 3
        

class Academico(Docente, Investigador):
    
    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Esta es la parte incorrecta, pero sigue los contenidos de herencia
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre
        self.oficina = oficina

        
p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación
ERROR! Session/line number was not unique in database. History logging moved to new session 6


En este ejemplo, como la clase `Academico` hereda tanto de `Docente` como de `Investigador`, parece natural llamar a ambos métodos `__init()__` de cada una de sus clases superiores. Sin embargo, esto provoca problemas en modelos más complejos, particularmente el **problema del diamante**.

## Solucionando el problema del diamante

Recordemos que tendremos una jerarquía de diamante cada vez que tenemos más de un "camino" en la jerarquía desde la clase inferior a una clase superior.

Veamos qué ocurre cuando llamamos al método `llamar()` en ambas superclases desde una clase `SubClaseA`, **sin utilizar `super()`**.

In [3]:
class ClaseB:
    
    num_llamadas_B = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Clase B...")
        print("Finalizando llamada al metodo en Clase B...")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    
    num_llamadas_izq = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Subclase Izquierda...")
        ClaseB.llamar(self)
        print("Finalizando llamada al metodo en Subclase Izquierda...")
        self.num_llamadas_izq += 1


class SubClaseDerecha(ClaseB):
    
    num_llamadas_der = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Subclase Derecha...")
        ClaseB.llamar(self)
        print("Finalizando llamada al metodo en Subclase Derecha...")
        self.num_llamadas_der += 1


class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    
    num_llamadas_subA = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Subclase A...")        
        SubClaseIzquierda.llamar(self)
        SubClaseDerecha.llamar(self)
        print("Finalizando llamada al metodo en Subclase A...")
        self.num_llamadas_subA += 1


s = SubClaseA()
s.llamar()
print()
print(f"Llamadas en Subclase A: {s.num_llamadas_subA}")
print(f"Llamadas en Subclase Izquierda: {s.num_llamadas_izq}")
print(f"Llamadas en Subclase Derecha: {s.num_llamadas_der}")
print(f"Llamadas en Clase B: {s.num_llamadas_B}")

Iniciando llamada al metodo en Subclase A...
Iniciando llamada al metodo en Subclase Izquierda...
Iniciando llamada al metodo en Clase B...
Finalizando llamada al metodo en Clase B...
Finalizando llamada al metodo en Subclase Izquierda...
Iniciando llamada al metodo en Subclase Derecha...
Iniciando llamada al metodo en Clase B...
Finalizando llamada al metodo en Clase B...
Finalizando llamada al metodo en Subclase Derecha...
Finalizando llamada al metodo en Subclase A...

Llamadas en Subclase A: 1
Llamadas en Subclase Izquierda: 1
Llamadas en Subclase Derecha: 1
Llamadas en Clase B: 2


Podemos apreciar que el método `llamar` de la clase de más arriba en la jerarquía (`ClaseB`) fue llamada dos veces. Luego de cada ejecución de `llamar`, la secuencia de invocaciones sube por la jerarquía hasta el método correspondiente en `ClaseB`.

La estructura de jerarquía en forma de diamante ocurre **siempre** que tengamos una clase que hereda de dos clases, aun cuando no tengamos una tercera superclase explícita. ¿Por qué? Porque en Python (y en varios lenguajes OOP), existe una clase superior [`object`](https://docs.python.org/3.6/library/functions.html#object) de la cual heredan **todas** las clases que creamos. 

En particular, cuando se utiliza multiherencia, podemos crear un diagrama, para visualizar estos casos, que se vería de la siguiente forma:

![Diamante2](img/diamante_2_small.png)

De esta manera, si, estando en un objeto de `SubClase`, llamamos al método `__init__` tanto de `ClaseA` como de `ClaseB`, estaríamos inicializando dos veces la clase `object`. Eso es precisamente lo que ocurre en el ejemplo de clase `Academico`, la cual llama explícitamente al inicializador de `Investigador` y de `Docente`, lo que tiene como consecuencia que la clase `object` se inicializa dos veces.


### Solución

La solución es que cada clase debe preocuparse de llamar a inicializar a la clase que la "precede" en el orden del esquema de la multiherencia, lo que puede hacerse llamando a `super()`. En Python, cada jerarquía posee un orden predefinido por la construcción de la jerarquía. El orden de las clases va **de izquierda a derecha** dentro de la lista de superclases desde donde hereda la subclase. 

En el siguiente ejemplo, basado en la primera jerarquía de diamante que presentamos, sólo debemos preocuparnos de hacer una llamada a `super()`. Python se encargará de que la llamada corresponda a la clase que respeta el orden en la multiherencia.

In [4]:
class ClaseB:
    
    num_llamadas_B = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Clase B...")
        print("Finalizando llamada al metodo en Clase B...")
        self.num_llamadas_B += 1


class SubClaseIzquierda(ClaseB):
    
    num_llamadas_izq = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Subclase Izquierda...")
        super().llamar()
        print("Finalizando llamada al metodo en Subclase Izquierda...")
        self.num_llamadas_izq += 1


class SubClaseDerecha(ClaseB):
    
    num_llamadas_der = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Subclase Derecha...")
        super().llamar()
        print("Finalizando llamada al metodo en Subclase Derecha...")
        self.num_llamadas_der += 1


class SubClaseA(SubClaseIzquierda, SubClaseDerecha):
    
    num_llamadas_subA = 0
    
    def llamar(self):
        print("Iniciando llamada al metodo en Subclase A...")
        super().llamar()
        print("Finalizando llamada al metodo en Subclase A...")
        self.num_llamadas_subA += 1


s = SubClaseA()
s.llamar()
print()
print(f"Llamadas en Subclase A: {s.num_llamadas_subA}")
print(f"Llamadas en Subclase Izquierda: {s.num_llamadas_izq}")
print(f"Llamadas en Subclase Derecha: {s.num_llamadas_der}")
print(f"Llamadas en Clase B: {s.num_llamadas_B}")

Iniciando llamada al metodo en Subclase A...
Iniciando llamada al metodo en Subclase Izquierda...
Iniciando llamada al metodo en Subclase Derecha...
Iniciando llamada al metodo en Clase B...
Finalizando llamada al metodo en Clase B...
Finalizando llamada al metodo en Subclase Derecha...
Finalizando llamada al metodo en Subclase Izquierda...
Finalizando llamada al metodo en Subclase A...

Llamadas en Subclase A: 1
Llamadas en Subclase Izquierda: 1
Llamadas en Subclase Derecha: 1
Llamadas en Clase B: 1


Podemos ver que esta vez estamos llamando solamente una vez al método `llamar` de `ClaseB`. Es más, si bien el diamante sigue existiendo, esta vez bastó ejecutar solo una vez el método `llamar` de `SubclaseA`, y conseguimos ejecutar el método `llamar` de todas las clases, y solamente una vez cada uno. 

Si ponemos atención a la secuencia de llamados, pareciera que hay un _orden_ impuesto entre las clases que componen el diamante. Este orden es: primero `SubClaseA`, luego `SubClaseIzquierda`, a continuación `SubClaseDerecha`, y finalmente `ClaseB`.

### Obteniendo el orden de herencia: el método `__mro__` 

La solución para determinar en qué orden se ejecutan los métodos en un esquema de multiherencia, se estableció mediante un algoritmo llamado [**C3**](https://www.python.org/download/releases/2.3/mro/) que permite calcular un orden lineal entre las clases que participan del esquema. Este algoritmo puede ser ejecutado por todas las clases de Python usando el método predefinido **`__mro__`**, cuyo nombre viene de _method resolution order_. Este método nos muestra el orden en la jerarquía de clases a partir de la clase actual. Es útil para casos de multiherencia complejos.

In [5]:
SubClaseA.__mro__

(__main__.SubClaseA,
 __main__.SubClaseIzquierda,
 __main__.SubClaseDerecha,
 __main__.ClaseB,
 object)

Notemos que el resultado de `__mro__` depende de la clase a la cual se aplica. Si lo aplicamos a `SubClaseIzquierda`, entonces su MRO no incluye a `SubClaseDerecha` pues ésta no es parte de su jerarquía.

In [6]:
SubClaseIzquierda.__mro__

(__main__.SubClaseIzquierda, __main__.ClaseB, object)

Ahora bien, **no toda estructura de multiherencia está permitida**. No es tan difícil armar una jerarquía en que no sea posible armar un MRO consistente para todas las clases. Por ejemplo:

In [7]:
class X():
    def call_me(self):
        print("soy X")
    
class Y():
    def call_me(self):
        print("soy Y")
    
class A(X, Y):
    def call_me(self):
        print("soy A")
    
class B(Y, X):
     def call_me(self):
        print("soy B")

class F(A, B):
    def call_me(self):
        print("soy F")

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y

En este ejemplo, tanto `A` como `B` heredan de `X` e `Y` pero en distinto orden. Esto no es un problema hasta que se define la clase `F` que hereda de `A` y de `B`. En este momento Python prohíbe la creación de la clase pues no puede determinar un MRO consistente para llegar a `X` e `Y`.

Para este caso, basta modificar `A` y `B` para que hereden de la misma manera, y ahora sí se puede determinar un MRO.

In [8]:
class X():
    def call_me(self):
        print("soy X")
    
class Y():
    def call_me(self):
        print("soy Y")
    
class A(X, Y):
    def call_me(self):
        print("soy A")
    
class B(X, Y):
     def call_me(self):
        print("soy B")

class F(A, B):
    def call_me(self):
        print("soy F")
        
print(F.__mro__)

(<class '__main__.F'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.X'>, <class '__main__.Y'>, <class 'object'>)


## Ejemplo multiherencia: clase `Academico`

Volviendo al ejemplo del académico que presentamos al inicio del tema de multiherencia, si bien el segundo código parece funcionar bien, ahora sabemos que si llamamos por separado a cada inicializador de las superclases, **estamos llamando dos veces al inicializador** de `object`, y no queremos que esto ocurra.

El código anterior era así:

In [9]:
class Investigador:

    def __init__(self, area):
        self.area = area
        # self.num_publicaciones = 0
        self.num_cursos = 3


class Docente:

    def __init__(self, departamento):
        self.departamento = departamento
        self.num_cursos = 0


class Academico(Docente, Investigador):

    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Queremos reemplazar esto por un super().__init__(...)
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre
        self.oficina = oficina


p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)
print(p1.num_cursos)

Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación
0


La solución, tal como lo hicimos con el método `llamar()` de la sección anterior, debería ser reemplazar las inicializaciones explícitas de `Investigador` y `Docente`, por un único llamado a `super().__init__()`, y así dejamos que MRO haga su trabajo.

Sin embargo, tenemos una situación levemente distinta. ¿Qué argumentos le entregamos a `super().__init__()`? Si le entregamos solamente los de `Investigador` o solamente los de `Docente`, uno de los inicializadores se quedaría sin argumentos.

In [10]:
class Investigador:

    def __init__(self, area):
        print("Inicializando investigador")
        self.area = area
        self.num_publicaciones = 0


class Docente:

    def __init__(self, departamento):
        print("Inicializando docente")
        self.departamento = departamento
        self.num_cursos = 3


class Academico(Docente, Investigador):

    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Solo un llamado, pero solo con un argumento
        super().__init__(departamento)
        self.nombre = nombre
        self.oficina = oficina


print(Academico.__mro__)
p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

(<class '__main__.Academico'>, <class '__main__.Docente'>, <class '__main__.Investigador'>, <class 'object'>)
Inicializando docente
Emilia Donoso


AttributeError: 'Academico' object has no attribute 'area'

Si bien, hay un MRO definido para `Academico`, cuando solo entregamos los argumentos de un inicializador, solo se ejecuta el `__init()__` de `Docente`, y no el de `Investigador`, por lo tanto nuestro `Academico` se queda sin su atributo `area`. 

Entonces una mejor solución sería entregarle a `super().__init()__` _todos_ los argumentos.

In [11]:
class Investigador:

    def __init__(self, area):
        print("Inicializando investigador")
        self.area = area
        self.num_publicaciones = 0


class Docente:

    def __init__(self, departamento):
        print("Inicializando docente")
        self.departamento = departamento
        self.num_cursos = 3


class Academico(Docente, Investigador):

    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Solo un llamado, con todos los argumentos que tenemos
        super().__init__(departamento, area_investigacion)
        self.nombre = nombre
        self.oficina = oficina


print(Academico.__mro__)
p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

(<class '__main__.Academico'>, <class '__main__.Docente'>, <class '__main__.Investigador'>, <class 'object'>)


TypeError: Docente.__init__() takes 2 positional arguments but 3 were given

Pero esto tampoco sirve, porque cada `__init__()` recibe solamente dos argumentos y le estamos entregando tres (no olvidemos al `self`). El ejemplo inicial con el método `llamar()` era muy ~~tramposo~~ sencillo porque `llamar()` no tenía argumentos. Estamos en un dilema.

### Solución: uso de `*args` y `**kwargs`

El dilema que tenemos se produce porque, aunque entreguemos todos los argumentos a `super().__init__()`, ninguno de los inicializadores sabe cuáles argumentos son para él, y cuáles para otro inicializador. Pero Python provee una solución a través de `*args` y `**kwargs`. Recordando:

* `**kwargs` es una *secuencia de argumentos de largo variable*, donde cada elemento de la lista tiene asociado un ***keyword***. El `**` mapea los elementos contenidos en el diccionario `kwargs` y los pasa a la función como _argumentos no posicionales_. Esto significa que los argumentos no se asignan a la función por su posición en el orden en que se entregan (como es lo habitual) sino por su _keyword_ asociado.
* `*args`, es una lista de argumentos de largo variable, pero sin *keywords* asociados. El operador `*` desempaqueta el contenido de args y los pasa a la función como argumentos posicionales. La función asigna valores a sus argumentos a partir del orden que trae esta lista.

### Ejemplo: solución para clase `Académico`

Podemos aplicar el uso de `*args` y `**kwargs` para, finalmente, implementar correctamente la inicialización en el ejemplo del académico. 

Recordemos el código que **queremos mejorar**:

In [12]:
class Investigador:

    def __init__(self, area):
        self.area = area
        self.num_publicaciones = 0


class Docente:

    def __init__(self, departamento):
        self.departamento = departamento
        self.num_cursos = 3


class Academico(Docente, Investigador):

    def __init__(self, nombre, oficina, area_investigacion, departamento):
        # Queremos reemplazar esto por un super().__init__(...), pero no sabemos qué argumentos usar
        Investigador.__init__(self, area_investigacion)
        Docente.__init__(self, departamento)
        self.nombre = nombre
        self.oficina = oficina


p1 = Academico("Emilia Donoso", "O-5", "Inteligencia de Máquina", "Ciencia De La Computación")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

Emilia Donoso
Inteligencia de Máquina
Ciencia De La Computación


Deseamos reemplazar los llamados a ambos inicializadores, por una única invocación `super().__init__()`, pero no sabemos qué argumentos entregar.

Aprovecharemos el hecho que `**kwargs` nos permite entregar un diccionario de argumentos.

In [13]:
class Investigador:

    def __init__(self, area, **kwargs):
        print(f"init Investigador con area '{area}' y kwargs:{kwargs}")
        super().__init__(**kwargs)
        self.area = area
        self.num_publicaciones = 0


class Docente:

    def __init__(self, departamento, **kwargs):
        print(f"init Docente con depto '{departamento}' y kwargs:{kwargs}")
        super().__init__(**kwargs)
        self.departamento = departamento
        self.num_cursos = 3


class Academico(Docente, Investigador):

    def __init__(self, nombre, oficina, **kwargs):
        print(f"init Academico con nombre '{nombre}', oficina '{oficina}', kwargs:{kwargs}")
        super().__init__(**kwargs)
        self.nombre = nombre
        self.oficina = oficina


print(Academico.__mro__)
print("--------")

p1 = Academico(
    "Emilia Donoso",
    oficina="O5",
    area="I.A.",
    departamento="Computación"
)
print("--------")
print(p1.nombre)
print(p1.area)
print(p1.departamento)

(<class '__main__.Academico'>, <class '__main__.Docente'>, <class '__main__.Investigador'>, <class 'object'>)
--------
init Academico con nombre 'Emilia Donoso', oficina 'O5', kwargs:{'area': 'I.A.', 'departamento': 'Computación'}
init Docente con depto 'Computación' y kwargs:{'area': 'I.A.'}
init Investigador con area 'I.A.' y kwargs:{}
--------
Emilia Donoso
I.A.
Computación


Este ejemplo permite, finalmente, llamar **una sola vez** y de manera correcta a todos los inicializadores de las clases de una jerarquía con multiherencia. El diccionario `**kwargs` contiene los argumentos identificados por su nombre, de manera que en cada llamado a un inicializador, éste extrae los _keywords_ que correspondan a algún nombre de los argumentos que espera, y el resto permanecen en el `**kwargs` y son pasados como argumento a la siguiente clase en la jerarquía. De esta manera, cada inicializador _consume_ del `**kwargs` lo que necesita. Notemos que en la última clase de la jerarquía, antes de llamar a `object`, todos los _keywords_ en `**kwargs` han sido consumidos, lo que está bien porque el inicializador de `object` no recibe argumentos.

Es muy **importante** incluir los operadores `*` y `**` cuando se ocupa multiherencia y cada clase padre recibe argumentos distintos en el `__init__`.

Una adecuada comprensión de los llamados a métodos/funciones usando lista variables de argumentos es muy práctica y flexible. Lo estaremos ocupando en numerosas ocasiones.

## Ejemplo multiherencia: clase `Avatar`

A continuación vamos a modelar al `Avatar`, un maestro de los 4 elementos. Para esto, vamos a crear 4 clases `MaestroXXXX` con su método `saludar()` que permitirá identificar al maestro y un método para hacer uso de su elemento.

In [14]:
class MaestroFuego:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro fuego")
        
    def fuego_control(self):
        print("¡Fuego control!")

        
class MaestroAgua:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro agua")
    
    def agua_control(self):
        print("¡Agua control!")

            
class MaestroTierra:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro tierra")

    def tierra_control(self):
        print("¡Tierra control!")
        
        
class MaestroAire:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro aire")
        
    def aire_control(self):
        print("¡Aire control!")

Ahora creamos a la clase `Avatar` que es un maestro en los 4 elementos. Así que hereda de los 4 maestros.

In [15]:
class Avatar(MaestroFuego, MaestroAgua, MaestroTierra, MaestroAire):
    def __init__(self, nombre: str):
        super().__init__(nombre)

In [16]:
el_ultimo_maestro_aire = Avatar("Aang")
el_ultimo_maestro_aire.saludar()

Hola!! mi nombre es Aang y soy un maestro fuego


**🤔 ¿Qué pasó? ¿Por qué no se imprimió los 4 saludos si es el avatar?**

Esto pasa porque el `Avatar` hereda de las 4 clases, y cada clase tiene el mismo método que es `saludar`. En este caso, se quedará el método **de la primera clase que heredamos**.

Intentemos utilizar `super()` para que se gatille el método de sus padres.

In [17]:
class Avatar(MaestroFuego, MaestroAgua, MaestroTierra, MaestroAire):
    def __init__(self, nombre: str):
        super().__init__(nombre)
        
    def saludar(self):
        super().saludar()
        
el_ultimo_maestro_aire = Avatar("Aang")
el_ultimo_maestro_aire.saludar()

Hola!! mi nombre es Aang y soy un maestro fuego


**🤔 ¿Qué pasó? ¿Por qué `super()` no solucionó esto?**

Es importante destacar que `super()` llama a la clase madre según el MRO, y se asegura que esta clase madre solo sea llamada 1 vez en todas las herencias, con el fín de solucionar el problema del diamante. Por lo tanto, si heredamos de múltiples clases, y cada una tiene el mismo método, cuando invoquemos el `super()`, este tambien llamará al método de la primera clase que heredamos.


Si exploramos el MRO de la clase `Avatar`, podremos ver el orden en el que serán llamados las clases cuando recurramos a `super()`.

In [18]:
Avatar.__mro__

(__main__.Avatar,
 __main__.MaestroFuego,
 __main__.MaestroAgua,
 __main__.MaestroTierra,
 __main__.MaestroAire,
 object)

**🤔 Y ¿hay formas de que se ejecuten el saludo de los 4 maestros?**

Si, una forma **NO RECOMENDADA** es agregar `super().saludar()` dentro de los 3 primeras clases `MaestroXXXXX` de las que estamos heredando, es decir, agregar `super().saludar()` en las clases `MaestroFuego`, `MaestroAgua` y `MaestroTierra`. Con esto, vamos a provocar que cada método de `saludar` llame al siguiente método `saludar()` dentro del MRO.

No agregaremos `super().saludar()` dentro de la clase `MaestroAire` porque si usamos `super()`, estaríamos accediendo a la clase `object` (recordar que toda clase hereda de `object`). Y esta no tiene el método `saludar`.

Esta forma no es recomendada porque estamos modificando 3 clases pequeñas (`MaestroFuego`, `MaestroAgua` y `MaestroTierra`) para que `Avatar` funcione, pero esto nos va a generar 2 problemas que explicaremos después de ver el código.

In [19]:
# Versión 2 - Usando super() dentro de los 3 primeros maestros

class MaestroFuego:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        super().saludar() # Agregar super() para forzar que se llame al siguiente saludar dentro del MRO
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro fuego")
        
    def fuego_control(self):
        print("¡Fuego control!")

        
class MaestroAgua:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        super().saludar() # Agregar super() para forzar que se llame al siguiente saludar dentro del MRO
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro agua")
    
    def agua_control(self):
        print("¡Agua control!")

            
class MaestroTierra:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        super().saludar() # Agregar super() para forzar que se llame al siguiente saludar dentro del MRO
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro tierra")

    def tierra_control(self):
        print("¡Tierra control!")
        
        
class MaestroAire:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        # No agregamos super porque esto llamaría a object y esa clase no tiene saludar()
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro aire")
        
    def aire_control(self):
        print("¡Aire control!")     
        
        
class Avatar(MaestroFuego, MaestroAgua, MaestroTierra, MaestroAire):
    def __init__(self, nombre: str):
        super().__init__(nombre)
        
    def saludar(self):
        super().saludar()

In [20]:
la_sucesora_de_aang = Avatar("Korra")
la_sucesora_de_aang.saludar()

Hola!! mi nombre es Korra y soy un maestro aire
Hola!! mi nombre es Korra y soy un maestro tierra
Hola!! mi nombre es Korra y soy un maestro agua
Hola!! mi nombre es Korra y soy un maestro fuego


Si bien logramos imprimir los 4 saludos. Surgen 2 problemas:

**Problema 1**: esta solución solo funciona cuando `MaestroAire` es la última clase que heredamos. Vamos a cambiar el orden de las herencias para a ver qué imprime.

In [21]:
class Avatar(MaestroFuego, MaestroAire, MaestroAgua, MaestroTierra):
    def __init__(self, nombre: str):
        super().__init__(nombre)
        
    def saludar(self):
        super().saludar()
        
la_sucesora_de_aang = Avatar("Korra")
la_sucesora_de_aang.saludar()

Hola!! mi nombre es Korra y soy un maestro aire
Hola!! mi nombre es Korra y soy un maestro fuego


Esto ocurre porque primero se llama a `saludar()` del `MaestroFuego`, esta clase hace `super().saludar()` para ejecutar el método `saludar()` de la clase `MaestroAire`. Luego, este método no hace `super()` así que no se sigue llamando a los siguientes `saludar()` del MRO. Esto implica que la solución de usar `super()` solo funcionó bajo cierto orden.


**Problema 2**: esta solución no permite que funcionen correctamente las clases `MaestroFuego`, `MaestroAgua` y `MaestroTierra`. Vamos a probar este caso instanciando un objeto de clase `MaestroFuego`. Primero veamos su MRO y luego instanciemos el objeto.

In [22]:
MaestroFuego.__mro__

(__main__.MaestroFuego, object)

In [23]:
maestro_fuego = MaestroFuego("Zuko")
maestro_fuego.saludar()

AttributeError: 'super' object has no attribute 'saludar'

Lo que ocurrió en este caso, es que el `super()` del `MaestroFuego` va a llamar a la siguiente clase, que es `object`, y esta clase no tiene el método saludar. Por lo tanto, agregar el `super()` para que `Avatar` funcione provocó que las demás clases con `super()` ya no funcionen correctamente.

Una solución podría ser que todos los Maestros heredaran de una clase padre, por ejemplo, `Humano` y que esta tenga un método `saludar(): pass`. Con esto, los `super()` no provocarían problemas. No obstante, al final estamos creando código adicional y modificando otras clases solo para que una nueva clase, `Avatar`, funcione correctamente. 

Otra solución, que veremos a continuación, es prescindir del uso de `super()` para esta versión. En su reemplazo, vamos a llamar directamente a los métodos de las clases padres.

In [24]:
# Versión 3 - No usar super en cada Maestro, pero que Avatar llame al método de cada maestro.

class MaestroFuego:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro fuego")
        
    def fuego_control(self):
        print("¡Fuego control!")

        
class MaestroAgua:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro agua")
    
    def agua_control(self):
        print("¡Agua control!")

            
class MaestroTierra:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro tierra")

    def tierra_control(self):
        print("¡Tierra control!")
        
        
class MaestroAire:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro aire")
        
    def aire_control(self):
        print("¡Aire control!")

        
class Avatar(MaestroFuego, MaestroAgua, MaestroTierra, MaestroAire):
    def __init__(self, nombre: str):
        super().__init__(nombre)
        
    def saludar(self):
        MaestroFuego.saludar(self)
        MaestroAgua.saludar(self)
        MaestroTierra.saludar(self)
        MaestroAire.saludar(self)

In [25]:
el_antecesor_de_aang = Avatar("Roku")
el_antecesor_de_aang.saludar()

Hola!! mi nombre es Roku y soy un maestro fuego
Hola!! mi nombre es Roku y soy un maestro agua
Hola!! mi nombre es Roku y soy un maestro tierra
Hola!! mi nombre es Roku y soy un maestro aire


En esta forma, fue la clase `Avatar` la encargada de llamar al método saludar de cada clase padre. De este modo, no es necesario usar `super()` y evaluar el MRO de la clase para entender el orden de ejecución, y redujimos la cantidad de código a editar para hacer funcionar esta nueva clase.


----

## Reflexión


Con este ejemplo, estamos viendo una de las dificultades que tiene la multiherencia, que es la escalabilidad de código y que el uso de `super()` a veces puede ser la solución al problema del diamante, pero otras veces puede generar ciertas dificultades o necesidades de modificar código ya existente. Por lo tanto, siempre es recomendable reflexionar sobre cuándo usar multiherencia y el `super()`, y cómo modelar la posible solución.

De todas formas, si bien este ejemplo mostró una dificultad de la multiherencia, tambien tiene sus ventajas. Por ejemplo, que `Avatar` ganó acceso a 4 métodos que no tenía inicialmente, sino que eran de sus clases padres.

In [26]:
el_antecesor_de_aang.fuego_control()
el_antecesor_de_aang.agua_control()
el_antecesor_de_aang.tierra_control()
el_antecesor_de_aang.aire_control()

¡Fuego control!
¡Agua control!
¡Tierra control!
¡Aire control!


Para finalizar esta reflexión, el último código presentado mostraba 4 clases con sus métodos de saludar y `xxxx_control()`, y una clase Avatar que puede saludar y llamar a cualquier tipo de elemento control. No obstante, siempre es bueno recordar que la modelación puede ser distinta para cada programador. 

A continuación se muestra otra posible modelación a este problema. Donde cada `MaestroXXXX` tendrá el método `saludar` y un nuevo método llamado `elemento_control` en reemplazo de `xxxx_control` (por ejemplo `fuego_control`). Ahora, será el `Avatar` quien puede diferenciar estos 4 métodos. De esta forma, las 4 clases de `Maestro` son muy similares en los métodos que tienen.

In [27]:
class MaestroFuego:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro fuego")
        
    def elemento_control(self):
        print("¡Fuego control!")

        
class MaestroAgua:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro agua")
    
    def elemento_control(self):
        print("¡Agua control!")

            
class MaestroTierra:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro tierra")

    def elemento_control(self):
        print("¡Tierra control!")
        
        
class MaestroAire:
    def __init__(self, nombre: str):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola!! mi nombre es {self.nombre} y soy un maestro aire")
        
    def elemento_control(self):
        print("¡Aire control!")

        
class Avatar(MaestroFuego, MaestroAgua, MaestroTierra, MaestroAire):
    def __init__(self, nombre: str):
        super().__init__(nombre)
        
    def saludar(self):
        MaestroFuego.saludar(self)
        MaestroAgua.saludar(self)
        MaestroTierra.saludar(self)
        MaestroAire.saludar(self)
        
    def fuego_control(self):
        MaestroFuego.elemento_control(self)
        
    def agua_control(self):
        MaestroAgua.elemento_control(self)
        
    def tierra_control(self):
        MaestroTierra.elemento_control(self)
        
    def aire_control(self):
        MaestroAire.elemento_control(self)

In [28]:
kyoshi = Avatar("kyoshi")
kyoshi.saludar()
print()
kyoshi.fuego_control()
kyoshi.agua_control()
kyoshi.tierra_control()
kyoshi.aire_control()

Hola!! mi nombre es kyoshi y soy un maestro fuego
Hola!! mi nombre es kyoshi y soy un maestro agua
Hola!! mi nombre es kyoshi y soy un maestro tierra
Hola!! mi nombre es kyoshi y soy un maestro aire

¡Fuego control!
¡Agua control!
¡Tierra control!
¡Aire control!


Te invitamos a pensar en otras formas de modelar esta situacion, y reflexionar sobre las ventajas y desventajas de cada forma. Algunas preguntas que pueden servir para reflexionar son
- ¿Cuál permite agregar nuevas clases sin editar las anteriores?
- ¿Cuál no presentaría problema del diamante?
- ¿Estoy generando código nuevo desde cero o estoy modificando código existente?
- Si tengo subclases, ¿estas deben funcionar de forma independiente o solo son un medio para que las clases que heredan de estas funcionen correctamente?